In [2]:
# Downloading test dataset

# https://drive.google.com/file/d/1Ewm2Rf_IUuzpEo8HQhk43jij6aiI3P2l/view?usp=share_link

!pip install -U --no-cache-dir gdown --pre

!gdown 1Ewm2Rf_IUuzpEo8HQhk43jij6aiI3P2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From: https://drive.google.com/uc?id=1Ewm2Rf_IUuzpEo8HQhk43jij6aiI3P2l
To: /content/test_images_annotated.zip
100% 55.0M/55.0M [00:02<00:00, 25.0MB/s]


In [3]:
!unzip test_images_annotated.zip

Archive:  test_images_annotated.zip
  inflating: 00000_00000_00018_png_jpg.rf.94eef97cce8d7579aeb6116744dd0c9b.jpg  
  inflating: __MACOSX/._00000_00000_00018_png_jpg.rf.94eef97cce8d7579aeb6116744dd0c9b.jpg  
  inflating: 00000_00000_00028_png_jpg.rf.550725fdfe9a0fa54366d32052966527.jpg  
  inflating: __MACOSX/._00000_00000_00028_png_jpg.rf.550725fdfe9a0fa54366d32052966527.jpg  
  inflating: 00000_00001_00001_png_jpg.rf.51ba9b49fb5fa54b4251d1b8c50975fe.jpg  
  inflating: __MACOSX/._00000_00001_00001_png_jpg.rf.51ba9b49fb5fa54b4251d1b8c50975fe.jpg  
  inflating: 00000_00001_00003_png_jpg.rf.efec461f019990d93fa51261d48f51f3.jpg  
  inflating: __MACOSX/._00000_00001_00003_png_jpg.rf.efec461f019990d93fa51261d48f51f3.jpg  
  inflating: 00000_00001_00009_png_jpg.rf.c82ad20febf5833d26e363339b6b893c.jpg  
  inflating: __MACOSX/._00000_00001_00009_png_jpg.rf.c82ad20febf5833d26e363339b6b893c.jpg  
  inflating: 00000_00001_00011_png_jpg.rf.b13df12f630e42dd9ce9a5b67fc39ac1.jpg  
  inflating: __MAC

In [4]:
!rm -rf __MACOSX/

In [5]:
!mkdir test_images_annotated
!mv *.jpg test_images_annotated/

In [6]:
!rm *.zip

In [7]:
!ls

sample_data  test_images_annotated


In [8]:
!rm -rf sample_data

In [9]:
!ls

test_images_annotated


In [10]:
!git clone https://github.com/jacobgil/pytorch-grad-cam.git

Cloning into 'pytorch-grad-cam'...
remote: Enumerating objects: 1097, done.
remote: Total 1097 (delta 0), reused 0 (delta 0), pack-reused 1097
Receiving objects: 100% (1097/1097), 110.20 MiB | 22.21 MiB/s, done.
Resolving deltas: 100% (609/609), done.


In [11]:
!ls

pytorch-grad-cam  test_images_annotated


In [12]:
%cd pytorch-grad-cam/

/content/pytorch-grad-cam


In [13]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!pip install -U --no-cache-dir gdown --pre
# https://drive.google.com/file/d/1kY80uzb_pGtG9rZu6N9yUDw5bYll4Rrh/view?usp=share_link
!gdown 1kY80uzb_pGtG9rZu6N9yUDw5bYll4Rrh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1kY80uzb_pGtG9rZu6N9yUDw5bYll4Rrh
To: /content/pytorch-grad-cam/best.pt
100% 14.5M/14.5M [00:00<00:00, 25.5MB/s]


In [15]:
!pwd

/content/pytorch-grad-cam


In [16]:
%cd /content
!mv test_images_annotated /content/pytorch-grad-cam

/content


In [17]:
%cd pytorch-grad-cam

/content/pytorch-grad-cam


In [18]:
!ls

best.pt   pyproject.toml    setup.cfg		   tutorials
cam.py	  pytorch_grad_cam  setup.py		   usage_examples
examples  README.md	    test_images_annotated
LICENSE   requirements.txt  tests


In [19]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch    
import cv2
import numpy as np
import requests
import torchvision.transforms as transforms
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from PIL import Image

COLORS = np.random.uniform(0, 255, size=(80, 3))

def parse_detections(results):
    detections = results.pandas().xyxy[0]
    detections = detections.to_dict()
    boxes, colors, names = [], [], []

    for i in range(len(detections["xmin"])):
        confidence = detections["confidence"][i]
        if confidence < 0.2:
            continue
        xmin = int(detections["xmin"][i])
        ymin = int(detections["ymin"][i])
        xmax = int(detections["xmax"][i])
        ymax = int(detections["ymax"][i])
        name = detections["name"][i]
        category = int(detections["class"][i])
        color = COLORS[category]

        boxes.append((xmin, ymin, xmax, ymax))
        colors.append(color)
        names.append(name)
    return boxes, colors, names


def draw_detections(boxes, colors, names, img):
    for box, color, name in zip(boxes, colors, names):
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(
            img,
            (xmin, ymin),
            (xmax, ymax),
            color, 
            2)

        cv2.putText(img, name, (xmin, ymin - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2,
                    lineType=cv2.LINE_AA)
    return img



In [21]:
!pwd

/content/pytorch-grad-cam


In [22]:
!mkdir cam_outputs

In [27]:
import os
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# model = torch.hub.load('s4nyam/temp', 'best', pretrained=True)
model = torch.hub.load('ultralytics/yolov5', 'custom', 'best.pt')  # custom/local model

# model = torch.load("best.pt")
model.eval()
model.cpu()
target_layers = [model.model.model.model[-3]]

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-1 Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7088338 parameters, 0 gradients
Adding AutoShape... 


In [33]:
for each_image in os.listdir('test_images_annotated'):
  # img = np.array(Image.open(requests.get(image_url, stream=True).raw))
  filename = 'test_images_annotated/'+str(each_image)
  img = np.array(Image.open(filename))
  img = cv2.resize(img, (640, 640))
  rgb_img = img.copy()
  img = np.float32(img) / 255
  transform = transforms.ToTensor()
  tensor = transform(img).unsqueeze(0)
  results = model([rgb_img])
  boxes, colors, names = parse_detections(results)
  detections = draw_detections(boxes, colors, names, rgb_img.copy())
  img = Image.fromarray(detections)
  img.save('cam_outputs/'+str(each_image), 'JPEG')
  


In [34]:
%cd /content/pytorch-grad-cam/
!mv cam_outputs cam_outputs1
!mkdir cam_outputs2

/content/pytorch-grad-cam


In [35]:
for each_image in os.listdir('test_images_annotated'):
  filename = 'test_images_annotated/'+str(each_image)
  img = np.array(Image.open(filename))
  img = cv2.resize(img, (640, 640))
  rgb_img = img.copy()
  img = np.float32(img) / 255
  
  cam = EigenCAM(model, target_layers, use_cuda=False)
  grayscale_cam = cam(tensor)[0, :, :]
  cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
  img=Image.fromarray(cam_image)
  img.save('cam_outputs2/'+str(each_image), 'JPEG')

In [36]:
def renormalize_cam_in_bounding_boxes(boxes, colors, names, image_float_np, grayscale_cam):
    """Normalize the CAM to be in the range [0, 1] 
    inside every bounding boxes, and zero outside of the bounding boxes. """
    renormalized_cam = np.zeros(grayscale_cam.shape, dtype=np.float32)
    for x1, y1, x2, y2 in boxes:
        renormalized_cam[y1:y2, x1:x2] = scale_cam_image(grayscale_cam[y1:y2, x1:x2].copy())    
    renormalized_cam = scale_cam_image(renormalized_cam)
    eigencam_image_renormalized = show_cam_on_image(image_float_np, renormalized_cam, use_rgb=True)
    image_with_bounding_boxes = draw_detections(boxes, colors, names, eigencam_image_renormalized)
    return image_with_bounding_boxes



In [37]:
!mkdir cam_outputs3
!mkdir cam_outputs4

In [38]:
for each_image in os.listdir('test_images_annotated'):

  filename = 'test_images_annotated/'+str(each_image)
  img = np.array(Image.open(filename))
  img = cv2.resize(img, (640, 640))
  rgb_img = img.copy()
  img = np.float32(img) / 255
  boxes, colors, names = parse_detections(results)
  cam = EigenCAM(model, target_layers, use_cuda=False)
  grayscale_cam = cam(tensor)[0, :, :]
  renormalized_cam_image = renormalize_cam_in_bounding_boxes(boxes, colors, names, img, grayscale_cam)
  cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
  img1 = Image.fromarray(renormalized_cam_image)
  img2 = Image.fromarray(np.hstack((rgb_img, cam_image, renormalized_cam_image)))
  img1.save('cam_outputs3/'+str(each_image), 'JPEG')
  img2.save('cam_outputs4/'+str(each_image), 'JPEG')

In [39]:
%cd /content/
!zip -r /content/camdata.zip /content/pytorch-grad-cam
from google.colab import files
import time
string = str(int(time.time()))
os.rename("camdata.zip","exp_"+string+".zip")
files.download("/content/"+"exp_"+string+".zip")

Streaming output truncated to the last 5000 lines.
  adding: content/pytorch-grad-cam/test_images_annotated/00001_00022_00010_png_jpg.rf.407418ba2be34e6f402c85c7a059f33c.jpg (deflated 14%)
  adding: content/pytorch-grad-cam/test_images_annotated/00034_00011_00026_png_jpg.rf.c54e2da3411187eb9cef1fa4ebc0415b.jpg (deflated 3%)
  adding: content/pytorch-grad-cam/test_images_annotated/00033_00005_00029_png_jpg.rf.d775726442f47fb24bde73068cbce469.jpg (deflated 5%)
  adding: content/pytorch-grad-cam/test_images_annotated/00019_00002_00014_png_jpg.rf.6f582f93d8954548013c2a3445b96fae.jpg (deflated 5%)
  adding: content/pytorch-grad-cam/test_images_annotated/00014_00006_00012_png_jpg.rf.c257065f4241f147db1397ba3d345411.jpg (deflated 7%)
  adding: content/pytorch-grad-cam/test_images_annotated/00001_00064_00024_png_jpg.rf.089409144e5f4a2599603f56b7005520.jpg (deflated 5%)
  adding: content/pytorch-grad-cam/test_images_annotated/00019_00003_00000_png_jpg.rf.b446abe93b724b1683bb66dd7cba5c8a.jpg (de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>